学生証番号 :

名前 :

メールアドレス(法政大学) :

# B2-1 convolutional neural networks (CNN) 入門

B2実験では、深層学習技術の中核を担うConvolutional Neural networks (CNN) について学ぶ。

具体的なCNNの説明は配布資料に譲るが、CNNは2次元画像全体をそのまま入力として受け取り、認識に必要な特徴抽出をモデル自身で行い、優れた認識性能を示す。

まずB2-1では、CNNの挙動や結果について実際のコードを動かしながら学ぶ。

## Setup

In [ ]:
# [1-0]
import numpy as np
import math
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

## データの準備


In [ ]:
# [1-1]
# get_data関数の定義
# B1-2の時とは異なり画像で扱うためflatten（2次元データ＞ベクトル化の処理）は削除
# data_sizeは学習データの数を絞るために使用
def get_data_mnist(data_size=None):
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    # 0 ~ 1の範囲に正規化を行う
    X_train = X_train.astype(np.float32) / 255.0
    X_test = X_test.astype(np.float32) / 255.0
    X_train = np.expand_dims(X_train, -1)
    X_test = np.expand_dims(X_test, -1)
    if data_size is not None:
        X_train = X_train[:data_size]
        y_train = y_train[:data_size]
    return (X_train, y_train), (X_test, y_test)

In [ ]:
# [1-2]
(X_train, y_train), (X_test, y_test) = get_data_mnist()
print(f"X_train shape : {X_train.shape} ")
print(f"X_test shape  : {X_test.shape} ")
print(f"y_train shape : {y_train.shape} ")
print(f"y_test shape  : {y_test.shape} ")

## CNNモデルの構築

このモデルは、5x5の畳み込みフィルタを持つ畳み込み層を1層だけ持ち、そのあとに2x2のmaxpoolによるサイズの1/4化、(12x12のフィルタが32個)そのあとに、それらを1次元化して（4608次元）、20次元へ次元削減する全結合層(linear1)、さらに10次元に減らす全結合層(linear2)を介して出力層（10ノード）のきわめてシンプルなCNNモデルである。


<img src="https://drive.google.com/uc?export=view&id=1xtvxqBtA0qeHe3JwyGtS27yBWxyC_IR9" width = 80%></img>


4608-20-10次元の部分は、先ほどの3層BPNNと同じである。

入力のチャネル$C$が1なのは、grayscale画像だから。
カラー画像だとR,G,Bの3になる。

In [ ]:
# [1-3]
# KerasによるCNNの定義
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(20, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

model.summary()

学習前に、得られている畳み込みフィルタを可視化してみる。

5x5(x1)のフィルタが32個ある。

In [ ]:
# [1-4]
# フィルターを可視化するための関数
# modelには可視化したいモデルを、layer_idには何番目のconv層のfilterを見るかのパラメータを渡す。
def plot_conv_filter(model, layer_id):
    conv_layers = [layer for layer in model.layers if "conv" in layer.name]
    if layer_id >= len(conv_layers):
        print(f"{layer_id} conv layer is not found")
        return None
    target_layer = conv_layers[layer_id]
    filter, bias = target_layer.get_weights()
    print(f"layer name : {target_layer.name}")
    print(f"filter shape : {filter.shape}")
    print(f"bias shape : {bias.shape}")
    num_filter = filter.shape[-1]

    cols = 8
    rows = math.ceil(num_filter / cols)
    idx = 1
    fig = plt.figure(figsize=(cols, rows))
    for i in range(rows):
        for j in range(cols):
            if idx > num_filter:
                continue
            ax = fig.add_subplot(rows, cols, idx, xticks=[], yticks=[])
            ax.imshow(filter[:, :, 0, idx - 1], cmap="gray")
            idx += 1
    plt.show()

In [ ]:
# [1-5]
# 1層目のフィルターの可視化
plot_conv_filter(model, 0)

まだ学習を行っていないため、ランダムな値になっている。

## モデルのコンパイルと初期値の性能確認



In [ ]:
# [1-6]
# 誤差関数と最適化手法を設定する関数を作成する。
def model_setup(model, optimizer="adam", lr=0.001):
    if optimizer == "adam":
        optim = keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = keras.optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optim, metrics=["accuracy"]
    )

In [ ]:
# [1-7]
# モデルのセットアップ。
model_setup(model)

# 学習前のモデルをテストデータで評価する
loss_no_train, accuracy_no_train = model.evaluate(X_test, y_test, verbose=0)
print("学習前の誤差 : ", loss_no_train)
print("学習前の正解率 : ", accuracy_no_train)

## CNNモデルの学習


In [ ]:
# [1-8]
# 学習を実行する。
# パラメータの設定
batch_size = 128
epochs = 10
# 学習回数は上で10回と定義している。
trainlog = model.fit(
    X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1
)

## 得られた結果のログから学習曲線を描画する。

In [ ]:
# [1-9]
# 学習曲線を描画する関数の作成
def plot_result(log):
    fig = plt.figure(figsize=(10, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(log.history["accuracy"])
    ax1.plot(log.history["val_accuracy"])
    ax1.set_title("Model accuracy")
    ax1.set_ylabel("Accuracy")
    ax1.set_xlabel("Epoch")
    ax1.legend(["Train", "Validation"], loc="best")
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(log.history["loss"])
    ax2.plot(log.history["val_loss"])
    ax2.set_title("Model loss")
    ax2.set_ylabel("Loss")
    ax2.set_xlabel("Epoch")
    ax2.legend(["Train", "Validation"], loc="best")
    plt.show()

In [ ]:
# [1-10]
# 実際に学習曲線の確認
# 右が正解率を左が誤差の推移を表している。
plot_result(trainlog)

## 学習後の畳み込みフィルタの可視化



In [ ]:
# [1-11]
# 学習後のフィルターの可視化
plot_conv_filter(model, layer_id=0)

初期値のフィルタに比べて、濃淡がはっきりと分かれるようになった。

## テストデータに対する性能の評価

In [ ]:
# [1-12]
# 評価
loss_train, accuracy_train = model.evaluate(X_test, y_test, verbose=0)

# 学習前のスコアの表示
print("学習前の誤差 : ", loss_no_train)
print("学習前の正解率 : ", accuracy_no_train)

# 学習後のスコアの表示
print("===== 学習後 =====")
print(f"学習後の誤差 :  ", loss_train)
print(f"学習後の正解率 : ", accuracy_train)

学習が終わってきちんと精度よく識別できることが確認できた。
（この課題はBPNNでもできていた）

## モデルが予測を間違えたデータの確認。


In [ ]:
# [1-13]
# データを表示する関数の作成
# Xにはデータをyにはそれに対応する正解ラベルを渡す
def random_plot(X, y, predict=None):
    W = 10
    H = 5
    fig = plt.figure(figsize=(10, 20))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=0.3, hspace=0.10, wspace=0.10)
    for i in range(5):
        for j in range(10):
            x_tmp = X[y == j]
            idx = np.random.randint(len(x_tmp))
            x = x_tmp[idx].reshape(28, 28)
            ax = fig.add_subplot(H, W, (i * 10) + j + 1, xticks=[], yticks=[])
            ax.imshow(x, cmap="gray")
            if predict is not None:
                pred_tmp = predict[y == j]
                p = pred_tmp[idx]
                ax.set_title(f"{j} -> {p}")
            else:
                ax.set_title(f"label : {j}")

In [ ]:
# [1-14]
# モデルの予測を取得。
test_predict = np.argmax(model.predict(X_test), 1)
# 間違えたものに絞って表示
false_idx = y_test != test_predict
# 間違えたもののみを描画
random_plot(X_test[false_idx], y_test[false_idx], test_predict[false_idx])

## 課題 B2-1-1 CNNの学習後のフィルタの考察
学習によって畳み込みフィルタは、どうしてこのようなパターン（[1-11]を参照）になったのかについて考えよ。

## 課題 B2-1-2 BPNN(前回のB1-2)との比較
データ数を減らした状態で(data_size=600)CNNのモデルを学習させ、性能を確認せよ。また、BPNNの時と比べて性能はどうか。またその差はなぜ起こったと考えらえるか。


ヒント :
- データ数を変更する場合は以下のコードを使用（データを上書きしてしまうので嫌なら変数名を変え、学習に渡すときも注意する）
```
data_size = 600
(X_train, y_train), _ = get_data_mnist(data_size=data_size)
```
- モデル構造や学習率の設定
```
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),     
        layers.Dense(10, activation="softmax"),
    ]
)
# 定義したモデルを確認
model.summary()
# 学習率の変更
lr = 0.001
model_setup(model, lr=lr)
```
- 学習のパラメータ(バッチサイズやエポック数)を設定
```
epochs=50
batch_size = 128
log = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
```
- 評価するとき
```
loss, accuracy = model.evaluate(X_test, y_test)
print("誤差　: ", loss)
print("正解率 : ", accuracy)
```

In [ ]:
# ここから実験を追加する。(自由にセルを増やしてOK)